In [6]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [7]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    y = np.zeros(vec.shape[0])
    y[0] = np.linalg.norm(vec)
    u = (vec - y)/np.linalg.norm(vec - y)
    H = np.eye(vec.shape[0]) - 2 * np.outer(u,u)
    outvec = y
    return outvec, H

Test your function using tests below:

In [8]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)
print(h)

assert_allclose(np.dot(h, v1), v, atol = 1e-15)
assert_allclose(np.dot(h, v), v1, atol = 1e-15)

[[ 0.26726124  0.53452248  0.80178373]
 [ 0.53452248  0.61007346 -0.5848898 ]
 [ 0.80178373 -0.5848898   0.12266529]]


In [9]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec, atol = 1e-10)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [251]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a.shape
    q = np.eye(m)
    a1_ = a1.copy()
    vec = np.array([])
    k = 0
    j = 0
    while j < min(n,m-1):
        k += 1
        H_ = np.eye(m)
        vec = a1[:,0]
        vec_, H = householder(vec)
        H_[j:,j:] = H
        #print(H_)
        a1_ = H_ @ a1_
        q = q @ H_.T
         
        a1 = a1_[j+1:,j+1:]
        j+=1
        #print(a1_)
    return q, a1_

        
    


In [252]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [253]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [254]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)
assert_allclose(q, qq)
assert_allclose(r, rr)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatch: 80%
Max absolute difference: 1.24996836
Max relative difference: 2.66316363
 x: array([[ 0.13665 ,  0.536013, -0.093698,  0.769714,  0.304596],
       [ 0.560359,  0.09354 , -0.533269,  0.018395, -0.626525],
       [ 0.197259,  0.659489,  0.600685, -0.323847, -0.245895],...
 y: array([[-0.13665 ,  0.536013,  0.093698,  0.661619, -0.497491],
       [-0.560359,  0.09354 ,  0.533269, -0.524772, -0.342763],
       [-0.197259,  0.659489, -0.600685, -0.37879 ,  0.147848],...

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

We can see, that our answer differ from library, because in that QR, we think about argument of $x_j$, to give this decompose uniqueness. We didn't think about this in our decompose.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [255]:
def qr_decomp_mod(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a.shape
    a1_ = a1.copy()
    vec = np.array([])
    M = np.array([])
    k = 0
    j = 0
    vec_ = np.ones(m)
    while j < min(n,m-1):
        k += 1
        vec = a1[:,0]
        y = np.zeros(vec.shape[0])
        y[0] = np.linalg.norm(vec)
        u = (vec - y)/np.linalg.norm(vec - y)
        a1_[j:,j:] = a1_[j:,j:] - 2*np.outer(u,u.T @ a1_[j:,j:])
        M = np.concatenate((M, u), axis = 0)
        a1 = a1_[j+1:,j+1:]
        j+=1

    return M, a1_

In [256]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)
ref, r1 = qr_decomp_mod(a)
print(ref)
# test the decomposition on upper triangular matrix, but we can know, that rounding, can replace 0 -> -0, and it can give to us mismatch
print(abs(r - r1))

[-0.6570196   0.42644006  0.15011669  0.47562065  0.37111197 -0.52846942
  0.73983285 -0.10990213  0.40160796 -0.79133207  0.36468006 -0.49071581]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [257]:
def r_constr(a,M):
    m, n = a.shape
    j = 0
    q = np.eye(m)
    q_ = np.eye(m)
    while j < min(m-1,n):
        Q_ = np.eye(m)
        u = M[-(m - min(n,m-1)+j+1):]
        M = M[0:(M.shape[0] - min(n,m-1) - j)]
        #print(u)
        q = np.eye(u.shape[0]) - 2 * np.outer(u,u)
        Q_[-q.shape[0]:,-q.shape[0]:] = q
        #print(Q_)
        q_ = q_ @ Q_
        j += 1
    return q_, q_ @ a


In [258]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)
ref, r1 = qr_decomp_mod(a)
q2, r2 = r_constr(a, ref)
print(abs(r- r1)) #checking absolute difference between upper triangular matrices from the first and the second decomposes
print(abs(r- r2)) #checking absolute difference between upper triangular matrices from the first decompose and constructing
print(abs(r1 - r2)) #checking absolute difference between upper triangular matrices from the second decompose and constructing
print(abs(q - q2.T)) #checking absolute difference of orthogonal matrices

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [260]:
"I am always check an absolute difference, because all algorithms can lost signes, when they get rid of computational errors."

'I am always check an absolute difference, because all algorithms can lost signes, when they get rid of computational errors.'